In [36]:
import itertools
import operator
import re
from abc import *
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft


def dict_to_str(dictionary: dict, space: int = 0):
    tab = lambda space: '  '*space
    string = f'{{\n'
    space += 1
    for key, value in zip(dictionary.keys(), dictionary.values()):
        if not isinstance(key, str):
            key = f'{key}'
        if '\n' in key:
            key = key.replace('\n', '')
        string += f'{tab(space)}{key}: '
        if not isinstance(value, dict):
            string += f'{value},\n'
        else:
            string += dict_to_str(value, space+1)
    string += f'{tab(space - 1)}}}\n'
    return string


def sparse_graph(g: nx.Graph) -> nx.Graph:
    con = nx.to_numpy_matrix(g)
    out = nx.from_numpy_matrix(
        lmd.suppress(
            pd.DataFrame(
                con
            ),
            trigger=con.mean().mean(),
            optimal=0
        ).to_numpy()
    )
    mapping = {node: label_name for node, label_name in zip(out, g)}
    out = nx.relabel_nodes(out, mapping)
    return out


def graph_to_connectome(g: nx.Graph) -> pd.DataFrame:
    return pd.DataFrame(
        nx.to_numpy_matrix(g),
        index= g.nodes,
        columns = g.nodes
    )


def labels_for_hemispheres(g: nx.Graph) -> Tuple[List[str], List[str]]:
    labels_rh, labels_lh = list(), list()
    for node in g.nodes:
        if 'lh' in node:
            labels_lh.append(node)
        elif 'rh' in node:
            labels_rh.append(node)
        else:
            raise ValueError(f'Wrong node name: {node}')
    return labels_lh, labels_rh


def hemispheres_division_modularity(g: nx.Graph) -> float:
    labels_lh, labels_rh = labels_for_hemispheres(g)
    return nx.algorithms.community.quality.modularity(g, [labels_lh, labels_rh])


def hemispheres_division_performance(g: nx.Graph) -> float:
    labels_lh, labels_rh = labels_for_hemispheres(g)
    return nx.algorithms.community.quality.performance(g, [labels_lh, labels_rh])


def graph_to_hemispheres(g: nx.Graph) -> Tuple[nx.Graph, nx.Graph]:
    labels_lh, labels_rh = labels_for_hemispheres(g)
    return g.subgraph(labels_lh), g.subgraph(labels_rh)


def smallworldness(g: nx.Graph) -> Tuple[float, float]:
    return nx.algorithms.smallworld.sigma(g), nx.algorithms.smallworld.omega(g)


def s_for_hemispheres(subjects: List[Subject]) -> pd.DataFrame:
    dataset = pd.DataFrame()

    for subject in subjects:
        resected_hemisphere = None
        lupd, rupd = dict(), dict()

        for node in subject.nodes:
            if node.type == 'resected' and 'rh' in node.label.name:
                lupd.update({'resected': False})
                rupd.update({'resected': True})
                break

            elif node.type == 'resected' and 'lh' in node.label.name:
                lupd.update({'resected': True})
                rupd.update({'resected': False})
                break

        for freq in subject.graph:
            for method in subject.graph[freq]:
                G = sparse_graph(subject.graph[freq][method])
                lh, rh = graph_to_hemispheres(G)
                lupd.update({f's_for_{method}': nx.algorithms.smetric.s_metric(lh, normalized=False)/100000})
                rupd.update({f's_for_{method}': nx.algorithms.smetric.s_metric(rh, normalized=False)/100000})

        dataset = append_series(dataset, pd.Series(lupd), index=f'{subject.name}_lh')
        dataset = append_series(dataset, pd.Series(rupd), index=f'{subject.name}_rh')

    return dataset


def metric_for_hemispheres(subjects: List[Subject], metric: Callable, **kwargs) -> pd.DataFrame:
    dataset = pd.DataFrame()

    for subject in subjects:
        start = time.time()
        resected_hemisphere = None
        lupd, rupd = dict(), dict()

        for node in subject.nodes:
            if node.type == 'resected' and 'rh' in node.label.name:
                lupd.update({'resected': False})
                rupd.update({'resected': True})
                break

            elif node.type == 'resected' and 'lh' in node.label.name:
                lupd.update({'resected': True})
                rupd.update({'resected': False})
                break

        for freq in subject.graph:
            for method in subject.graph[freq]:
                G = sparse_graph(subject.graph[freq][method])
                lh, rh = graph_to_hemispheres(G)
                lupd.update({f'{metric.__name__}_for_{method}': metric(lh, **kwargs)})
                rupd.update({f'{metric.__name__}_for_{method}': metric(rh, **kwargs)})

        dataset = append_series(dataset, pd.Series(lupd), index=f'{subject.name}_lh')
        dataset = append_series(dataset, pd.Series(rupd), index=f'{subject.name}_rh')
        print(f'{subject.name}: DONE, RUNTIME: {time.time() - start}')

    return dataset

class NodesInformativeness(object):
    def __init__(self):
        self._subjective, self._anatomical = tuple(
            {
                'correct': {'true': dict(), 'false': dict()},
                'wrong': {'true': dict(), 'false': dict()}
            }
            for _ in range(2)
        )

    @property
    def subjective(self):
        return self._subjective

    @property
    def anatomical(self):
        return self._anatomical

    @property
    def informativeness(self):
        return self._subjective, self._anatomical

    @informativeness.setter
    def informativeness(self, sample: Tuple[str, bool, str]):
        sample_name, positive, group = sample
        if positive:
            subjective_container = self._subjective[group]['true']
            anatomical_container = self._anatomical[group]['true']
        else:
            subjective_container = self._subjective[group]['false']
            anatomical_container = self._anatomical[group]['false']
        if sample_name in subjective_container:
            subjective_container[sample_name] += 1
            anatomical_container[sample_name[5:]] += 1
        else:
            subjective_container.update({sample_name: 1})
            anatomical_container.update({sample_name[5:]: 1})

    @anatomical.setter
    def anatomical(self, value: Any):
        raise PermissionError('Impossible to set value to anatomical')

    @subjective.setter
    def subjective(self, value: Any):
        raise PermissionError('Impossible to set value to subjective')

    def sorted(self, container_name: str, prediction: str, positive: bool, type: str = 'growth', reverse: bool = False):

        def rating_sort(container_name: str, prediction: str, key: str) -> Callable:

            contrary = 'correct' if prediction == 'wrong' else 'wrong'

            def sort_tool(item: Tuple[str, int]) -> Union[int, float]:
                mistakes = self.__dict__['_' + container_name][contrary][key][item[0]] + 1 if item[0] in self.__dict__['_' + container_name][contrary][key] else 1
                return item[1]/mistakes

            return sort_tool

        if type == 'growth':
            return dict(
                sorted(
                    self.__dict__
                    ['_' + container_name]
                    [prediction]
                    [str(positive).lower()]
                        .items(),
                    key=lambda item: item[1],
                    reverse=reverse)
            )
        elif type == 'rating':
            return dict(
                sorted(
                    self.__dict__
                    ['_' + container_name]
                    [prediction]
                    [str(positive).lower()]
                        .items(),
                    key=rating_sort(container_name, prediction, str(positive).lower()),
                    reverse=reverse)
            )

class SubjectInformativeness(object):
    def __init__(self):
        self._informativeness = {
                'correct': {'true': dict(), 'false': dict()},
                'wrong': {'true': dict(), 'false': dict()}
            }

    @property
    def informativeness(self):
        return self._informativeness

    @informativeness.setter
    def informativeness(self, sample: Tuple[str, bool, str]):
        sample_name, positive, group = sample
        subject_name = sample_name[:4]
        if positive:
            container = self._informativeness[group]['true']
        else:
            container = self._informativeness[group]['false']
        if subject_name in container:
            container[subject_name] += 1
        else:
            container.update({subject_name: 1})

    def sorted(self, prediction: str, positive: bool, type: str = 'growth', reverse: bool = False):

        def rating_sort(prediction: str, key: str) -> Callable:

            contrary = 'correct' if prediction == 'wrong' else 'wrong'

            def sort_tool(item: Tuple[str, int]) -> Union[int, float]:
                mistakes = self._informativeness[contrary][key][item[0]] + 1 if item[0] in self._informativeness[contrary][key] else 1
                return item[1]/mistakes

            return sort_tool

        if type == 'growth':
            return dict(
                sorted(
                    self._informativeness
                    [prediction][str(positive).lower()].items(),
                    key=lambda item: item[1],
                    reverse=reverse)
            )
        elif type == 'rating':
            return dict(
                sorted(
                    self._informativeness
                    [prediction][str(positive).lower()].items(),
                    key=rating_sort(prediction, str(positive).lower()),
                    reverse=reverse)
            )

In [ ]:
# subjects = pipeline(
#     methods=['wpli', 'envelope'],
#     freq_bands=(7.5, 12),
#     centrality_metrics=['eigen', 'close', 'between', 'degree', ],# 'katz', 'info', 'harmonic']
#     subjects_specificity={
#         'M2S2': {
#             'freq_bands': (7.5, 12.5)
#         },
#         'R1D2': {
#             'freq_bands': (7.5, 11)
#         },
#         'S1A2': {
#             'freq_bands': (5, 10)
#         },
#         'S1H1': {
#             'freq_bands': (8, 13)
#         },
#         'K1V1': {
#             'freq_bands': (7.5, 11)
#         },
#         'L1P1': {
#             'freq_bands': (5, 10)
#         },
#         'M1G2': {
#             'freq_bands': (7, 11)
#         },
#         'G1V2': {
#             'freq_bands': (7, 11)
#         },
#         'G1R1': {
#             'freq_bands': (12.5, 16.5)
#         },
#         'M1N2': {
#             'freq_bands': (10, 15)
#         },
#         'B1R1': {
#             'freq_bands': (6, 11)
#         },
#         'B1C2': {
#             'freq_bands': (7.5, 12.5)
#         },
#         'J1T2': {
#             'freq_bands': (11, 15)
#         },
#         'O1O2': {
#             'freq_bands': (5.5, 9.5)
#         },
#     }
# )

# subjects = pipeline(
#     methods=['wpli', 'envelope'],
#     freq_bands=(4, 8),
#     centrality_metrics=['eigen', 'close', 'between', 'degree']
#     )

subjects = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

# for subject in subjects:
#     for dataset in subject.datasets:
#         print(subject.datasets[dataset].head())
        # print(dataset)
        # print(subject.dataset[dataset])
        # columns = subject.dataset[dataset].columns.to_list()
        # rule = dict()
        # for column in columns:
        #     if 'wpli' in column:
        #         rule.update({column: 'wpli'})
        #     if 'envelope' in column:
        #         rule.update({column: 'envelope'})
        # subject.dataset[dataset] = subject.dataset[dataset].rename(columns=rule, copy=True)



datasets = {
    subject: {
        freq: {
            method: lmd.suppress(
                    subject.connectomes[freq][method],
                    trigger=subject.connectomes[freq][method].mean().mean(),
                    optimal=0
                )
            # method: subject.connectomes[freq][method]
           for method in subject.connectomes[freq]
        }
        for freq in subject.connectomes
    }
    for subject in subjects
}

In [ ]:
close, between, eigen, degree, info, harmony = dict(), dict(), dict(), dict(), dict(), dict()
for subject in datasets:
    close.update({subject.name: dict()})
    between.update({subject.name: dict()})
    eigen.update({subject.name: dict()})
    degree.update({subject.name: dict()})
    info.update({subject.name: dict()})
    harmony.update({subject.name: dict()})
    # katz.update({subject.name: dict()})
    for freq in datasets[subject]:
        for method in datasets[subject][freq]:
            print(subject.name, freq, method)
            label_names = datasets[subject][freq][method].columns
            if nx.is_connected(
                    nx.convert_matrix.from_numpy_array(
                        datasets[subject][freq][method].to_numpy()
                    )
            ):
                arr  = datasets[subject][freq][method].to_numpy()
            elif nx.is_connected(
                    nx.convert_matrix.from_numpy_array(
                        subject.connectomes[freq][method].to_numpy()
                    )
            ):
                arr = subject.connectomes[freq][method].to_numpy()
            else:
                raise ValueError(f'Graph not connected: {subject.name}, {freq}, {method}')
            if arr.min().min() < 0:
                for i in range(arr.shape[0]):
                    for j in range(arr.shape[1]):
                        if arr[i, j] != 0: arr[i, j] -= arr.min().min()
            G = nx.convert_matrix.from_numpy_array(arr)
            mapping = {node: label_name for node, label_name in zip(G, label_names)}
            G = nx.relabel_nodes(G, mapping)
            for place, data in zip(
                [
                    close[subject.name],
                    between[subject.name],
                    eigen[subject.name],
                    info[subject.name],
                    harmony[subject.name],
                    degree[subject.name],
                    # katz[subject.name]
                ],[
                        nx.closeness_centrality(G, distance='weight'),
                        nx.betweenness_centrality(G, weight='weight'),
                        nx.eigenvector_centrality_numpy(G, weight='weight'),
                        nx.information_centrality(G, weight='weight'),
                        nx.harmonic_centrality(G, distance='weight'),
                        dict(G.degree(weight='weight')),
                        # nx.katz_centrality(G, weight='weight', max_iter=100)
                    ]
            ):
                place.update({
                    freq + '_' + method: pd.Series(data)
                })

print('All is done')

In [ ]:
datasets_centrality = dict()

for subject in subjects:
    datasets_centrality.update({subject.name: dict()})
    datasets_centrality[subject.name].update({
        'close': pd.DataFrame(close[subject.name]),
        'between': pd.DataFrame(between[subject.name]),
        'eigen': pd.DataFrame(eigen[subject.name]),
        'degree': pd.DataFrame(degree[subject.name]),
        'info': pd.DataFrame(info[subject.name]),
        'harmony': pd.DataFrame(harmony[subject.name]),
        # 'katz': pd.DataFrame(katz[subject.name])
    })
    true = subject.datasets['eigen']['resected']
    datasets_centrality[subject.name]['close'] = datasets_centrality[subject.name]['close'].assign(resected=true)
    datasets_centrality[subject.name]['between'] = datasets_centrality[subject.name]['between'].assign(resected=true)
    datasets_centrality[subject.name]['eigen'] = datasets_centrality[subject.name]['eigen'].assign(resected=true)
    datasets_centrality[subject.name]['degree'] = datasets_centrality[subject.name]['degree'].assign(resected=true)
    datasets_centrality[subject.name]['info'] = datasets_centrality[subject.name]['info'].assign(resected=true)
    datasets_centrality[subject.name]['harmony'] = datasets_centrality[subject.name]['harmony'].assign(resected=true)
    # datasets_centrality[subject.name]['katz'] = datasets_centrality[subject.name]['katz'].assign(resected=true)

for subject in subjects:
    subject.datasets = datasets_centrality[subject.name]

In [ ]:
stat1 = SubjectsStatistic(subjects, 'resected', centrality_metric='eigen')
print('done 1')
stat2 = SubjectsStatistic(subjects, 'resected', centrality_metric='between')
print('done 2')
stat3 = SubjectsStatistic(subjects, 'resected', centrality_metric='close')
print('done 3')
stat4 = SubjectsStatistic(subjects, 'resected', centrality_metric='degree')
print('done 4')
stat5 = SubjectsStatistic(subjects, 'resected', centrality_metric='info')
print('done 5')
stat6 = SubjectsStatistic(subjects, 'resected', centrality_metric='harmony')
print('done 6')
# stat7 = SubjectsStatistic(subjects, 'resected', centrality_metric='katz')
print('ok')

In [ ]:
test1 = stat1.test(state='reflected')
test2 = stat2.test(state='reflected')
test3 = stat3.test(state='reflected')
test4 = stat4.test(state='reflected')
test5 = stat5.test(state='reflected')
test6 = stat6.test(state='reflected')
test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples = list(), list(), list(), list(), list(), list()
for feature in test1.result:
    test1_samples.append(test1.result[feature][1])
    test2_samples.append(test2.result[feature][1])
    test3_samples.append(test3.result[feature][1])
    test4_samples.append(test4.result[feature][1])
    test5_samples.append(test5.result[feature][1])
    test6_samples.append(test6.result[feature][1])

test_samples = np.array([
    np.array(test1_samples),
    np.array(test2_samples),
    np.array(test3_samples),
    np.array(test4_samples),
    np.array(test5_samples),
    np.array(test6_samples)
])

df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
    'eigen', 'between', 'close', 'degree', 'info', 'harmony'
]).T
print(df)
df.to_csv('/home/user/Downloads/Wilcoxon.csv')

In [ ]:
test1 = stat1.test(state='reflected', test='mannwhitneyu')
test2 = stat2.test(state='reflected', test='mannwhitneyu')
test3 = stat3.test(state='reflected', test='mannwhitneyu')
test4 = stat4.test(state='reflected', test='mannwhitneyu')
test5 = stat5.test(state='reflected', test='mannwhitneyu')
test6 = stat6.test(state='reflected', test='mannwhitneyu')
test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples = list(), list(), list(), list(), list(), list()
for feature in test1.result:
    test1_samples.append(test1.result[feature][1])
    test2_samples.append(test2.result[feature][1])
    test3_samples.append(test3.result[feature][1])
    test4_samples.append(test4.result[feature][1])
    test5_samples.append(test5.result[feature][1])
    test6_samples.append(test6.result[feature][1])

test_samples = np.array([
    np.array(test1_samples),
    np.array(test2_samples),
    np.array(test3_samples),
    np.array(test4_samples),
    np.array(test5_samples),
    np.array(test6_samples)
])

df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
    'eigen', 'between', 'close', 'degree', 'info', 'harmony'
]).T
print(df)
df.to_csv('/home/user/Downloads/MannWhitneyu.csv')

In [ ]:
# plt.imshow(
#     pd.DataFrame(
#         nx.to_numpy_matrix(
#             subjects[0].graph['7.5-12.5Hz']['wpli']
#         )
#     ).to_numpy())
# plt.show()
# plt.imshow(
#     lmd.suppress(
#         pd.DataFrame(
#             nx.to_numpy_matrix(
#                 subjects[0].graph['7.5-12.5Hz']['wpli']
#             )
#         ),
#         optimal=0
#     ).to_numpy())
# plt.show()
#
G = sparse_graph(subjects[0].graph['7.5-12.5Hz']['wpli'])
lh, rh = graph_to_hemispheres(G)
print(lh.number_of_edges(), lh.number_of_nodes())
# nx.draw(subjects[0].graph['7.5-12.5Hz']['wpli'])
# plt.show()
# nx.draw(G)
# plt.show()
# nx.draw(lh)
# plt.show()
# nx.draw(rh)
# plt.show()
# print('all: ', smallworldness(G))
import time
start = time.time()
# print(smallworldness(nx.complete_graph(5)))
# print(time.time() - start)
# print(smallworldness(nx.complete_graph(10)))
# print(time.time() - start)
# print(smallworldness(nx.complete_graph(75)))
# print(time.time() - start)
# print('lh: ', smallworldness(lh))
# print('rh: ', smallworldness(rh))

In [ ]:
for subject in subjects:
    print(subject.name)
    print('\n\tresected nodes:')
    for node in subject.nodes:
        if node.type == 'resected':
            print(f'\t{node.label.name}')
    for freq in subject.graph:
        for method in subject.graph[freq]:
            print(f'\n\t{freq}: {method}')
            G = sparse_graph(subject.graph[freq][method])
            lh, rh = graph_to_hemispheres(G)
            print(f'\n\themispheres division modularity: {hemispheres_division_modularity(G)}\n')
            print('\n\t s for lh', nx.algorithms.smetric.s_metric(lh, normalized=False)/100000)
            print('\n\t s for rh', nx.algorithms.smetric.s_metric(rh, normalized=False)/100000)

In [ ]:
# dataset = metric_for_hemispheres(subjects, nx.algorithms.cluster.transitivity)
dataset = metric_for_hemispheres(subjects, nx.algorithms.global_efficiency)
# dataset = s_for_hemispheres(subjects)
print(dataset)

In [ ]:
y = dataset['resected'].to_numpy()
x = dataset.drop(['resected'], axis=1).to_numpy()

scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = MaxAbsScaler()
# scaler = RobustScaler()
x = scaler.fit_transform(x)

acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()

for i in range(100):

    x_train, x_test, y_train, y_test = train_test_split(x, y)

    # clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10)
    # clf = AdaBoostClassifier(n_estimators=10)
    clf = svm.SVC()
    # clf = svm.SVC(kernel='linear', class_weight={True: .8, False: 1})
    # clf = svm.SVC(kernel='sigmoid', class_weight={True: .8, False: 1})
    # clf = svm.SVC(class_weight={True: .8, False: 1})
    # clf = SGDClassifier()
    # clf = KNeighborsClassifier(n_neighbors=3)
    # clf = LogisticRegression(class_weight={True: .8, False: 1})
    # clf = RandomForestClassifier(max_depth=20)
    # clf = GaussianNB()
    # clf = LinearDiscriminantAnalysis()
    # clf = QuadraticDiscriminantAnalysis()
    # clf = KMeans(n_clusters=2, algorithm='full')
    # clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10), max_iter=1450)

    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    acc.append(accuracy_score(y_test, pred))
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    spec.append(tn / (tn + fp))
    sens.append(tp / (tp + fn))

    if np.isnan(tp/(tp + fp)):
        pospred.append(0)
    else:
        pospred.append(tp/(tp + fp))

    if np.isnan((tn/(tn + fn))):
        negpred.append(0)
    else:
        negpred.append(tn/(tn + fn))

max_acc = np.max(np.array(acc))
min_acc = np.min(np.array(acc))
moda_acc, cacc = sp.stats.mode(np.array(acc))
acc = sum(acc)/len(acc)
max_spec = np.max(np.array(spec))
min_spec = np.min(np.array(spec))
moda_spec, cspec = sp.stats.mode(np.array(spec))
spec = sum(spec)/len(spec)
max_sens = np.max(np.array(sens))
min_sens = np.min(np.array(sens))
moda_sens, csens = sp.stats.mode(np.array(sens))
sens = sum(sens)/len(sens)
max_pospred = np.max(np.array(pospred))
min_pospred = np.min(np.array(pospred))
moda_pospred, cpospred = sp.stats.mode(np.array(pospred))
pospred = sum(pospred)/len(pospred)
max_negpred = np.max(np.array(negpred))
min_negpred = np.min(np.array(negpred))
moda_negpred, cnegspred = sp.stats.mode(np.array(negpred))
negpred = sum(negpred)/len(negpred)
print('Accuracy: ', acc, min_acc, max_acc, moda_acc, cacc)
print('Specificity', spec, min_spec, max_spec, moda_spec, cspec)
print('Sensitivity: ', sens, min_sens, max_sens, moda_sens, csens)
print('Positive Predictive Value: ', pospred, min_pospred, max_pospred, moda_pospred, cpospred)
print('Negative Predictive Value: ', negpred, min_negpred, max_negpred, moda_negpred, cnegspred)

In [ ]:
for subject in subjects:
    print(subject.name)
    print('\n\tresected nodes:')
    for node in subject.nodes:
        if node.type == 'resected':
            print(f'\t{node.label.name}')
    for freq in subject.graph:
        for method in subject.graph[freq]:
            print(f'\n\t{freq}: {method}')
            G = sparse_graph(subject.graph[freq][method])
            c = graph_to_connectome(G)
            lh, rh = graph_to_hemispheres(G)
            print(f'\n\themispheres division modularity: {hemispheres_division_modularity(G)}\n')
            comp = nx.algorithms.community.centrality.girvan_newman(G)
            for c in next(comp):
                print(f'\t{c}')

In [37]:
# features = ['0.5-4Hz_wpli', '4-7Hz_wpli', '7-14Hz_wpli', '14-30Hz_wpli', '30-70Hz_wpli']
# features = ['14-30Hz_wpli', '4-7Hz_wpli']
# features = ['4-7Hz_wpli', '0.5-4Hz_envelope', '4-7Hz_envelope', '7-14Hz_envelope', '14-30Hz_envelope', '30-70Hz_envelope']
# features = ['4-7Hz_wpli', '4-7Hz_envelope']
features = ['4-8Hz_wpli', '4-8Hz_envelope']
# features = ['4-8Hz_wpli2_debiased', '4-8Hz_envelope']
# features = ['4-8Hz_wpli']
# features = ['4-7Hz_envelope']
# features = ['4-7Hz_wpli', '4-7Hz_psd']
acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()
stat = stat1

stat.datasets['true']['4-8Hz_envelope'] = stat2.datasets['true']['4-8Hz_envelope']
stat.datasets['false_mirror']['4-8Hz_envelope'] = stat2.datasets['false_mirror']['4-8Hz_envelope']

nodes_informativeness = NodesInformativeness()
subject_informativeness = SubjectInformativeness()

for i in range(100):
    # clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10)
    # clf = AdaBoostClassifier(n_estimators=10)
    # clf = svm.SVC(class_weight={True: .9, False: 1})
    clf = svm.SVC(kernel='sigmoid')
    # clf = svm.SVC(class_weight={True: 1, False: .8}, probability=True)
    # clf = SGDClassifier()
    # clf = KNeighborsClassifier(n_neighbors=7, metric='chebyshev')
    # clf = LogisticRegression(class_weight={True: 1, False: .8})
    # clf = LogisticRegression()
    # clf = RandomForestClassifier(max_depth=20)
    # clf = GaussianNB()
    # clf = LinearDiscriminantAnalysis()
    # clf = QuadraticDiscriminantAnalysis()
    # clf = KMeans(n_clusters=2, algorithm='full')
    # clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10), max_iter=1450)
    true_data = stat.datasets['true'][features]
    # true_data = lmd.lead_std(stat.datasets['true'][features], take_std_from=stat1.datasets['false_res'][features], axis=1)
    # false_data = stat.random_samples()[features]
    # false_data = lmd.lead_std(stat.datasets['false_res'][features], take_std_from=stat1.datasets['true'][features], axis=1)
    # false_data = stat1.datasets['false_res'][features]
    # false_data = stat1.datasets['false'][features]
    # true_data = stat1.random_samples()[features]
    # false_data = stat.random_samples()[features]
    false_data = stat.datasets['false_mirror'][features]
    true_data = true_data.assign(resected=True)
    false_data = false_data.assign(resected=False)
    samples = [[sample] for sample in true_data.index.tolist() + false_data.index.tolist()]

    dataset = pd.concat([true_data, false_data], axis=0)
    dataset = dataset.sample(frac = 1)


    scaler = StandardScaler()
    # scaler = MinMaxScaler()
    # scaler = MaxAbsScaler()
    # scaler = RobustScaler()



    y = dataset['resected'].to_numpy()
    dataset = dataset.drop(['resected'], axis=1)

    # dataset = lmd.lead_std(dataset, take_std_from=stat1.datasets['false_res'][features], axis=1)
    # dataset = lmd.suppress(dataset, axis=1, optimal='max')
    # dataset = lmd.promote(dataset, axis=1, optimal='max')
    # dataset = lmd.clusterize(dataset, axis=1, n_clusters=3, optimal='symclose')
    # dataset = lmd.binarize(dataset, axis=1)
    x = scaler.fit_transform(dataset)
    x = np.append(x, samples, axis=1)
    # x = dataset
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    train_samples, test_samples = x_train[:, 2], x_test[:, 2]
    x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)

    for predicted, actual, sample in zip(pred, y_test, test_samples):
        nodes_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'
        subject_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'


    # prob = clf.predict_proba(x_test).tolist()
    # for p, x, y in zip(prob, pred, y_test):
    #     print(y, x, p)

    # df = pd.DataFrame(np.array([np.array(y_test), pred]).T, columns=['actually', 'prediction'])
    # print(df)

    acc.append(accuracy_score(y_test, pred))
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    spec.append(tn / (tn + fp))
    sens.append(tp / (tp + fn))

    if np.isnan(tp/(tp + fp)):
        pospred.append(0)
    else:
        pospred.append(tp/(tp + fp))

    if np.isnan((tn/(tn + fn))):
        negpred.append(0)
    else:
        negpred.append(tn/(tn + fn))

max_acc = np.max(np.array(acc))
min_acc = np.min(np.array(acc))
moda_acc, cacc = sp.stats.mode(np.array(acc))
acc = sum(acc)/len(acc)
max_spec = np.max(np.array(spec))
min_spec = np.min(np.array(spec))
moda_spec, cspec = sp.stats.mode(np.array(spec))
spec = sum(spec)/len(spec)
max_sens = np.max(np.array(sens))
min_sens = np.min(np.array(sens))
moda_sens, csens = sp.stats.mode(np.array(sens))
sens = sum(sens)/len(sens)
max_pospred = np.max(np.array(pospred))
min_pospred = np.min(np.array(pospred))
moda_pospred, cpospred = sp.stats.mode(np.array(pospred))
pospred = sum(pospred)/len(pospred)
max_negpred = np.max(np.array(negpred))
min_negpred = np.min(np.array(negpred))
moda_negpred, cnegspred = sp.stats.mode(np.array(negpred))
negpred = sum(negpred)/len(negpred)
print('Accuracy: ', acc, min_acc, max_acc, moda_acc, cacc)
print('Specificity', spec, min_spec, max_spec, moda_spec, cspec)
print('Sensitivity: ', sens, min_sens, max_sens, moda_sens, csens)
print('Positive Predictive Value: ', pospred, min_pospred, max_pospred, moda_pospred, cpospred)
print('Negative Predictive Value: ', negpred, min_negpred, max_negpred, moda_negpred, cnegspred)

print(
    dict_to_str(
        nodes_informativeness.anatomical
    )
)

# print(
#     dict_to_str(
#         nodes_informativeness.sorted(
#             'subjective', prediction='correct', positive=False, type='rating', reverse=True
#         )
#     )
# )
#
# print(
#     dict_to_str(
#         subject_informativeness.sorted(
#             prediction='correct', positive=False, type='rating', reverse=True
#         )
#     )
# )

Accuracy:  0.6379999999999999 0.45 0.85 [0.65] [23]
Specificity 0.6805296925296925 0.3333333333333333 1.0 [0.72727273] [8]
Sensitivity:  0.6097096514596515 0.25 0.9 [0.5] [12]
Positive Predictive Value:  0.6591116106116105 0.3 1.0 [0.5] [13]
Negative Predictive Value:  0.6380085747585751 0.2 0.9090909090909091 [0.5] [10]
{
  correct: {
      true: {
          G_temporal_inf-rh: 11,
          G_pariet_inf-Angular-rh: 9,
          G_front_sup-rh: 7,
          Pole_temporal-rh: 19,
          Pole_temporal-lh: 22,
          G_front_middle-rh: 13,
          G_pariet_inf-Supramar-rh: 13,
          S_front_middle-lh: 7,
          S_temporal_inf-rh: 14,
          G_temporal_inf-lh: 18,
          G_occipital_middle-lh: 12,
          G_temporal_middle-lh: 20,
          G_pariet_inf-Angular-lh: 10,
          G_front_inf-Opercular-rh: 15,
          G&S_occipital_inf-lh: 17,
          S_front_sup-rh: 9,
          G&S_occipital_inf-rh: 17,
          S_temporal_transverse-rh: 11,
          G_temp_sup

In [44]:
cross_anatomical_nodes_informativeness = dict()
cross_subjective_nodes_informativeness = dict()
cross_subject_informativeness = dict()

for _ in range(10):
    # features = ['0.5-4Hz_wpli', '4-7Hz_wpli', '7-14Hz_wpli', '14-30Hz_wpli', '30-70Hz_wpli']
    # features = ['14-30Hz_wpli', '4-7Hz_wpli']
    # features = ['4-7Hz_wpli', '0.5-4Hz_envelope', '4-7Hz_envelope', '7-14Hz_envelope', '14-30Hz_envelope', '30-70Hz_envelope']
    # features = ['4-7Hz_wpli', '4-7Hz_envelope']
    features = ['4-8Hz_wpli', '4-8Hz_envelope']
    # features = ['4-8Hz_wpli2_debiased', '4-8Hz_envelope']
    # features = ['4-8Hz_wpli']
    # features = ['4-7Hz_envelope']
    # features = ['4-7Hz_wpli', '4-7Hz_psd']
    acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()
    stat = stat1

    stat.datasets['true']['4-8Hz_envelope'] = stat2.datasets['true']['4-8Hz_envelope']
    stat.datasets['false_mirror']['4-8Hz_envelope'] = stat2.datasets['false_mirror']['4-8Hz_envelope']

    nodes_informativeness = NodesInformativeness()
    subject_informativeness = SubjectInformativeness()

    for i in range(100):
        # clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10)
        # clf = AdaBoostClassifier(n_estimators=10)
        # clf = svm.SVC(class_weight={True: .9, False: 1})
        clf = svm.SVC(kernel='sigmoid')
        # clf = svm.SVC(class_weight={True: 1, False: .8}, probability=True)
        # clf = SGDClassifier()
        # clf = KNeighborsClassifier(n_neighbors=7, metric='chebyshev')
        # clf = LogisticRegression(class_weight={True: 1, False: .8})
        # clf = LogisticRegression()
        # clf = RandomForestClassifier(max_depth=20)
        # clf = GaussianNB()
        # clf = LinearDiscriminantAnalysis()
        # clf = QuadraticDiscriminantAnalysis()
        # clf = KMeans(n_clusters=2, algorithm='full')
        # clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10), max_iter=1450)
        true_data = stat.datasets['true'][features]
        # true_data = lmd.lead_std(stat.datasets['true'][features], take_std_from=stat1.datasets['false_res'][features], axis=1)
        # false_data = stat.random_samples()[features]
        # false_data = lmd.lead_std(stat.datasets['false_res'][features], take_std_from=stat1.datasets['true'][features], axis=1)
        # false_data = stat1.datasets['false_res'][features]
        # false_data = stat1.datasets['false'][features]
        # true_data = stat1.random_samples()[features]
        # false_data = stat.random_samples()[features]
        false_data = stat.datasets['false_mirror'][features]
        true_data = true_data.assign(resected=True)
        false_data = false_data.assign(resected=False)
        samples = [[sample] for sample in true_data.index.tolist() + false_data.index.tolist()]

        dataset = pd.concat([true_data, false_data], axis=0)
        dataset = dataset.sample(frac = 1)


        scaler = StandardScaler()
        # scaler = MinMaxScaler()
        # scaler = MaxAbsScaler()
        # scaler = RobustScaler()



        y = dataset['resected'].to_numpy()
        dataset = dataset.drop(['resected'], axis=1)

        # dataset = lmd.lead_std(dataset, take_std_from=stat1.datasets['false_res'][features], axis=1)
        # dataset = lmd.suppress(dataset, axis=1, optimal='max')
        # dataset = lmd.promote(dataset, axis=1, optimal='max')
        # dataset = lmd.clusterize(dataset, axis=1, n_clusters=3, optimal='symclose')
        # dataset = lmd.binarize(dataset, axis=1)
        x = scaler.fit_transform(dataset)
        x = np.append(x, samples, axis=1)
        # x = dataset
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        train_samples, test_samples = x_train[:, 2], x_test[:, 2]
        x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)

        for predicted, actual, sample in zip(pred, y_test, test_samples):
            nodes_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'
            subject_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'


        # prob = clf.predict_proba(x_test).tolist()
        # for p, x, y in zip(prob, pred, y_test):
        #     print(y, x, p)

        # df = pd.DataFrame(np.array([np.array(y_test), pred]).T, columns=['actually', 'prediction'])
        # print(df)

        acc.append(accuracy_score(y_test, pred))
        tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
        spec.append(tn / (tn + fp))
        sens.append(tp / (tp + fn))

        if np.isnan(tp/(tp + fp)):
            pospred.append(0)
        else:
            pospred.append(tp/(tp + fp))

        if np.isnan((tn/(tn + fn))):
            negpred.append(0)
        else:
            negpred.append(tn/(tn + fn))

    for type in nodes_informativeness.anatomical:
        if type in cross_anatomical_nodes_informativeness:
            for positive in nodes_informativeness.anatomical[type]:
                if positive in cross_anatomical_nodes_informativeness[type]:
                    for sample in nodes_informativeness.anatomical[type][positive]:
                        if sample in cross_anatomical_nodes_informativeness[type][positive]:
                            cross_anatomical_nodes_informativeness[type][positive][sample] = np.append(
                                cross_anatomical_nodes_informativeness[type][positive][sample],
                                nodes_informativeness.anatomical[type][positive][sample]
                            )
                        else:
                            cross_anatomical_nodes_informativeness[type][positive].update(
                                {sample: np.array([nodes_informativeness.anatomical[type][positive][sample]])}
                            )
                else:
                    cross_anatomical_nodes_informativeness[type].update(
                        {
                            positive: {
                            sample: np.array([nodes_informativeness.anatomical[type][positive][sample]])
                            for sample in nodes_informativeness.anatomical[type][positive]
                            }
                        }
                    )
        else:
            cross_anatomical_nodes_informativeness.update(
                {
                    type: {
                        positive: {
                            sample: np.array([nodes_informativeness.anatomical[type][positive][sample]])
                            for sample in nodes_informativeness.anatomical[type][positive]
                        }
                        for positive in nodes_informativeness.anatomical[type]
                    }
                }
            )

    for type in nodes_informativeness.subjective:
        if type in cross_subjective_nodes_informativeness:
            for positive in nodes_informativeness.subjective[type]:
                if positive in cross_subjective_nodes_informativeness[type]:
                    for sample in nodes_informativeness.subjective[type][positive]:
                        if sample in cross_subjective_nodes_informativeness[type][positive]:
                            cross_subjective_nodes_informativeness[type][positive][sample] = np.append(
                                cross_subjective_nodes_informativeness[type][positive][sample],
                                nodes_informativeness.subjective[type][positive][sample]
                            )
                        else:
                            cross_subjective_nodes_informativeness[type][positive].update(
                                {sample: np.array([nodes_informativeness.subjective[type][positive][sample]])}
                            )
                else:
                    cross_subjective_nodes_informativeness[type].update(
                        {
                            positive: {
                            sample: np.array([nodes_informativeness.subjective[type][positive][sample]])
                            for sample in nodes_informativeness.subjective[type][positive]
                            }
                        }
                    )
        else:
            cross_subjective_nodes_informativeness.update(
                {
                    type: {
                        positive: {
                            sample: np.array([nodes_informativeness.subjective[type][positive][sample]])
                            for sample in nodes_informativeness.subjective[type][positive]
                        }
                        for positive in nodes_informativeness.subjective[type]
                    }
                }
            )

    for type in subject_informativeness.informativeness:
        if type in cross_subject_informativeness:
            for positive in subject_informativeness.informativeness[type]:
                if positive in cross_subject_informativeness[type]:
                    for sample in subject_informativeness.informativeness[type][positive]:
                        if sample in cross_subject_informativeness[type][positive]:
                            cross_subject_informativeness[type][positive][sample] = np.append(
                                cross_subject_informativeness[type][positive][sample],
                                subject_informativeness.informativeness[type][positive][sample]
                            )
                        else:
                            cross_subject_informativeness[type][positive].update(
                                {sample: np.array([subject_informativeness.informativeness[type][positive][sample]])}
                            )
                else:
                    cross_subject_informativeness[type].update(
                        {
                            positive: {
                            sample: np.array([subject_informativeness.informativeness[type][positive][sample]])
                            for sample in subject_informativeness.informativeness[type][positive]
                            }
                        }
                    )
        else:
            cross_subject_informativeness.update(
                {
                    type: {
                        positive: {
                            sample: np.array([subject_informativeness.informativeness[type][positive][sample]])
                            for sample in subject_informativeness.informativeness[type][positive]
                        }
                        for positive in subject_informativeness.informativeness[type]
                    }
                }
            )

print(
    dict_to_str(
        cross_anatomical_nodes_informativeness
    )
)

print(
    dict_to_str(
        cross_subjective_nodes_informativeness
    )
)

print(
    dict_to_str(
        cross_subject_informativeness
    )
)

<ipython-input-44-94dc63807c73>:99: RuntimeWarning: invalid value encountered in long_scalars
  if np.isnan(tp/(tp + fp)):


{
  correct: {
      true: {
          G_front_middle-lh: [11 24  6 15 22 24 11 18 26 11],
          S_occipital_ant-lh: [9 9 7 8 7 9 9 5 8 8],
          S_circular_insula_inf-rh: [13 11  9  3  8  8 10  7 10  9],
          G_front_middle-rh: [19 14 29 19  8 25 21 22 16 20],
          G_temporal_middle-rh: [18  9 17 14 18 16 12 17 10 15],
          Pole_temporal-rh: [21 27 20 13 20 25 17 23 14 19],
          S_temporal_inf-lh: [13 19 14 13 18 15 15  9 15 10],
          G_temp_sup-Lateral-lh: [10  6  5  6  5 10  1  6  5  5],
          G_front_inf-Opercular-lh: [ 9 15 23 20 13 10 15 16 16 17],
          G_temporal_inf-lh: [16 18 13 12 12 12 13 12  5 11],
          G&S_occipital_inf-rh: [12 10 15 10  6 10 19 18 14 11],
          Pole_occipital-lh: [ 9  7  5  7  9  8  8 11  8  6],
          S_oc_sup&transversal-rh: [ 6  6  8  7 11  7  7  6 15 10],
          G_temporal_middle-lh: [11  9 14 13 11 14 14 11 13 15],
          S_front_sup-lh: [2 6 4 9 8 4 6 8 6 7],
          S_temporal_sup-lh: [2

In [50]:
def mean_and_std(arr: np.ndarray) -> Tuple[float, float]:
    return arr.mean(), arr.std()

def cross_statistic(statistic: Dict[str, Dict[str, Dict[str, np.ndarray]]]) -> Dict[str, Dict[str, Dict[str, Tuple[float, float]]]]:
    return {
        class_type: {
            pos : {
                sample:  mean_and_std(statistic[class_type][pos][sample])
                for sample in statistic[class_type][pos]
            } for pos in statistic[class_type]
        } for class_type in statistic
    }

cross_anatomical_nodes_informativeness_ = cross_statistic(cross_anatomical_nodes_informativeness)
cross_subjective_nodes_informativeness_ = cross_statistic(cross_subjective_nodes_informativeness)
cross_subject_informativeness_ = cross_statistic(cross_subject_informativeness)

def stat_sort(statistic):
    def rating_sort(item):
        key, value = item
        return \
            statistic['correct']['true'][key][0]*statistic['correct']['false'][key][0]/\
            statistic['correct']['true'][key][1]/statistic['correct']['false'][key][1]/\
            statistic['wrong']['true'][key][0]/statistic['wrong']['false'][key][0]/\
            statistic['wrong']['true'][key][1]/statistic['wrong']['false'][key][1]


    return {
        class_type: {
            pos :
                dict(
                    sorted(
                        statistic[class_type][pos].items(),
                        key=rating_sort,
                        reverse=True)
                    )
            for pos in statistic[class_type]
        } for class_type in statistic
    }

print(
    dict_to_str(
        stat_sort(cross_anatomical_nodes_informativeness_)
    )
)

print(
    dict_to_str(
        stat_sort(cross_subjective_nodes_informativeness_)
    )
)

print(
    dict_to_str(
        stat_sort(cross_subject_informativeness_)
    )
)

{
  correct: {
      true: {
          G_temp_sup-Plan_polar-lh: (7.4, 2.1071307505705477),
          Pole_occipital-lh: (7.8, 1.5999999999999999),
          G_orbital-rh: (7.7, 1.3453624047073711),
          G_oc-temp_lat-fusifor-rh: (6.4, 1.9078784028338913),
          G_occipital_middle-rh: (7.7, 2.794637722496424),
          S_interm_prim-Jensen-rh: (8.4, 2.009975124224178),
          S_circular_insula_inf-lh: (8.8, 2.0396078054371136),
          G_temp_sup-Plan_polar-rh: (7.9, 3.5902646142032486),
          Lat_Fis-post-rh: (8.5, 2.5787593916455256),
          S_circular_insula_ant-rh: (7.3, 2.1931712199461306),
          S_circular_insula_ant-lh: (7.5, 2.80178514522438),
          S_oc_sup&transversal-lh: (7.5, 2.8722813232690143),
          S_interm_prim-Jensen-lh: (8.9, 1.3),
          S_front_middle-lh: (7.1, 1.5779733838059502),
          G_front_sup-rh: (6.2, 1.8867962264113207),
          S_occipital_ant-lh: (7.9, 1.2206555615733703),
          Pole_occipital-rh: (8.8, 3.4)

In [ ]:
# features = ['0.5-4Hz_wpli', '4-7Hz_wpli', '7-14Hz_wpli', '14-30Hz_wpli', '30-70Hz_wpli']
# features = ['14-30Hz_wpli', '4-7Hz_wpli']
# features = ['4-7Hz_wpli', '4-7Hz_envelope']
# features = ['4-7Hz_wpli', '4-7Hz_envelope']
features = ['wpli', 'envelope']
# features = ['4-7Hz_wpli']
# features = ['envelope']
# features = ['4-7Hz_wpli', '4-7Hz_psd']
acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()
stat = stat1
for i in range(1000):
    # clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10)
    # clf = AdaBoostClassifier(n_estimators=10, random_state=0)
    # clf = svm.SVC(kernel='linear')
    clf = svm.SVC(class_weight={True: .9, False: 1}, probability=True)
    # clf = KNeighborsClassifier(n_neighbors=3)
    # clf = LogisticRegression()
    # clf = RandomForestClassifier(max_depth=20)
    # clf = GaussianNB()
    # clf = LinearDiscriminantAnalysis()
    # clf = KMeans(n_clusters=2, algorithm='full')
    # clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(10, 10), max_iter=1450)

    true_data = stat.datasets['true'][features]
    # false_data = stat1.random_samples()[features]
    # false_data = stat1.datasets['false_res'][features]
    # true_data = stat1.random_samples()[features]
    # false_data = stat.random_samples()[features]
    # false_data = lmd.lead_std(stat.datasets['false_res'][features], take_std_from=stat.datasets['true'][features], axis=1)
    false_data = stat.datasets['false_mirror'][features]
    true_data = true_data.assign(resected=True)
    false_data = false_data.assign(resected=False)
    dataset = pd.concat([true_data, false_data], axis=0)
    dataset = dataset.sample(frac = 1)


    scaler = StandardScaler()
    # scaler = MinMaxScaler()
    # scaler = MaxAbsScaler()
    # scaler = RobustScaler()



    y = dataset['resected'].to_numpy()
    dataset = dataset.drop(['resected'], axis=1)


    # dataset = lmd.lead_std(dataset, take_std_from=stat1.datasets['false_res'][features], axis=1)
    dataset = lmd.suppress(dataset, axis=1, optimal='max')
    # dataset = lmd.promote(dataset, axis=1, optimal='min')
    # dataset = lmd.clusterize(dataset, axis=1, n_clusters=3, optimal='max')
    # dataset = lmd.binarize(dataset, axis=1)


    x = scaler.fit_transform(dataset)
    # x = dataset
    scores = cross_val_score(clf, x, y, cv=10)
    acc.append(scores.mean())

print('Accuracy: ', sum(acc)/len(acc), min(acc), max(acc))

In [ ]:
# from nodestimation.mlearning.features import prepare_connectivity, prepare_data
# con = pkl.load(open(subjects[0].data['con'], 'rb'))

subjects_dir, subjects_ = find_subject_dir()
labels = mne.read_labels_from_annot('B1C2', parc='aparc', subjects_dir=subjects_dir)
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]

label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
# methods = [
#     'coh',
#     'imcoh',
#     'plv',
#     'ciplv',
#     'ppc',
#     'pli'
# ]
#
# freq_bands = [
#     '0.5-4Hz',
#     '4-7Hz',
#     '7-14Hz',
#     '14-30Hz'
# ]
#
# for method in methods:
#     for freq_band in freq_bands:
#         fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
#         mne.viz.plot_connectivity_circle(con[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
#                                          node_angles=node_angles, node_colors=node_colors,
#                                          title='All-to-All Connectivity Epilepsy Condition ({} for {})'
#                                          .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
#                                          fontsize_names=20, fig=fig
#                                          )

In [ ]:
for subject in subjects:
    fig, ax = plt.subplots(figsize=(15,15))
    display = nplt.plot_glass_brain(None, display_mode='lyrz', figure=fig, axes=ax)
    spared = [node.nilearn_coordinates for node in subject.nodes if node.type == 'spared']
    resected = [node.nilearn_coordinates for node in subject.nodes if node.type == 'resected']
    resection = read['resec-mni'](subject.data['resec-mni'])
    display.add_markers(resection, marker_color="violet", marker_size=1)
    display.add_markers(np.array(spared), marker_color="yellow", marker_size=100)
    display.add_markers(np.array(resected), marker_color="red", marker_size=250)



